<a href="https://colab.research.google.com/github/greercr02/fvc7fe_DS5111su24_Lab2_DatabaseDesignAndBuild/blob/main/fvc7fe_Data_Engineering_Lab2_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTRUCTIONS

The project is divided into two parts, and you will submit your work separately for each part.
The first project part involves understanding the problem and uses cases, and designing a relational database to support the needs. The second project part involves database buildout and query construction and testing.  You can use any of several drawing programs to create an Entity Relationship Diagram for the design section.  One such tool is dbdiagram.io, and another popular tool is LucidChart.com


Data: included in this project package



PART 1
Part 1 of the assignment is to understand the problem and design the database.
You will then answer the following Design Questions and submit your responses.

Part 1 Total Points: 20

PART 2
Part 2 of the assignment is to build the database, insert all data, and write SQL queries to answer use case questions. We will be using DBT to insert our initial seed data, and write SQL statement to transform it.  Instead of a database, we will use Snowflake to store the data.



Use Case Questions

After building and populating your database, you will write SQL queries to answer the questions below.
Include your SQL code and results in your submitted file.



10a) (2 PTS) Which courses had more than one instructor for the same term? Provide the mnemonic and term for each. Note this occurs in courses with multiple sections.

10b) (1 PT) For courses with multiple sections, provide the term, course mnemonic, and instructor name for each. Hint: You can use your result from 10a in a subquery or WITH clause.

Note: Question 10 is good preparation for SQL interview questions.



In [10]:
!pip install pandas


In [11]:
import pandas as pd
import sqlite3

# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()


In [12]:
# Load the Excel file
file_path = 'current_learning_objectives_raw_data.xlsx'
excel_data = pd.ExcelFile(file_path)

# List all sheet names
print("Sheet names:", excel_data.sheet_names)


Sheet names: ['assigned_raw', 'other_data_raw', 'tech_bootcamp', '5001', '5012', '5100', '5110', '6001', '6002', '6003', '6011', '6012', '6013', '6021', '6030', '6040', '6050', 'biz_analytics']


In [13]:
# Inspect the 'assigned_raw' sheet
assigned_raw = excel_data.parse('assigned_raw')
print(assigned_raw.head())

# Inspect one of the course sheets, e.g., '5001'
course_5001 = excel_data.parse('5001')
print(course_5001.head())


  TEACHING ASSIGNMENTS: INSTRUCTORS TO COURSES BY TERM  \
0                                                NaN     
1                                           FALL2021     
2                                   ds_tech_bootcamp     
3                                           stat6021     
4                                             ds5100     

                                  Unnamed: 1   Unnamed: 2  Unnamed: 3  \
0                                        NaN          NaN         NaN   
1                                        NaN          NaN         NaN   
2                           ds tech bootcamp  Pete Alonzi         NaN   
3  STAT 6021: Linear Models for Data Science  Jeffrey Woo         NaN   
4       DS 5100 Programming for Data Science     Judy Fox         NaN   

         Unnamed: 4                                 Unnamed: 5   Unnamed: 6  \
0               NaN                                        NaN          NaN   
1        SUMMER2021                                   

In [14]:
# Create tables
cursor.execute("""
CREATE TABLE courses (
    course_mnemonic TEXT PRIMARY KEY,
    course_name TEXT,
    is_active INTEGER
);
""")

cursor.execute("""
CREATE TABLE learning_outcomes (
    learning_outcome_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_mnemonic TEXT,
    description TEXT,
    is_active INTEGER,
    FOREIGN KEY (course_mnemonic) REFERENCES courses (course_mnemonic)
);
""")

cursor.execute("""
CREATE TABLE instructors (
    instructor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    is_active INTEGER
);
""")

cursor.execute("""
CREATE TABLE assignments (
    assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    course_mnemonic TEXT,
    instructor_id INTEGER,
    term TEXT,
    FOREIGN KEY (course_mnemonic) REFERENCES courses (course_mnemonic),
    FOREIGN KEY (instructor_id) REFERENCES instructors (instructor_id)
);
""")

print("Database schema created!")



Database schema created!


In [15]:
# Populate courses table
courses = [
    ('DS5001', 'Introduction to Data Science', 1),
    ('DS5100', 'Programming for Data Science', 1),
    ('DS6001', 'Practice of Data Science', 0)  # Example inactive course
]
cursor.executemany("INSERT INTO courses VALUES (?, ?, ?);", courses)
conn.commit()
print("Courses table populated")


Courses table populated


In [16]:
# Populate learning outcomes
learning_outcomes = [
    (None, 'DS5001', 'Learn fundamental concepts of data science', 1),
    (None, 'DS5100', 'Master Python programming for data science', 1),
    (None, 'DS6001', 'Develop real-world data science projects', 0)
]
cursor.executemany("INSERT INTO learning_outcomes VALUES (?, ?, ?, ?);", learning_outcomes)
conn.commit()
print("Learning outcomes table populated")


Learning outcomes table populated


In [17]:
# Populate instructors table
instructors = [
    (None, 'Alice Johnson', 1),
    (None, 'Bob Smith', 1),
    (None, 'Charlie Brown', 0)  # Example inactive instructor
]
cursor.executemany("INSERT INTO instructors VALUES (?, ?, ?);", instructors)
conn.commit()
print("Instructors table populated!")


Instructors table populated!


In [18]:
# Populate assignments table
assignments = [
    (None, 'DS5001', 1, 'Fall 2021'),
    (None, 'DS5100', 2, 'Summer 2021'),
    (None, 'DS6001', 3, 'Spring 2021')
]
cursor.executemany("INSERT INTO assignments VALUES (?, ?, ?, ?);", assignments)
conn.commit()
print("Assignments table populated!")


Assignments table populated!


In [19]:
query = "SELECT course_mnemonic, course_name FROM courses WHERE is_active = 1;"
print(pd.read_sql(query, conn))


  course_mnemonic                   course_name
0          DS5001  Introduction to Data Science
1          DS5100  Programming for Data Science


In [20]:
query = "SELECT course_mnemonic, course_name FROM courses WHERE is_active = 0;"
print(pd.read_sql(query, conn))


  course_mnemonic               course_name
0          DS6001  Practice of Data Science


In [21]:
query = "SELECT name FROM instructors WHERE is_active = 0;"
print(pd.read_sql(query, conn))


            name
0  Charlie Brown


In [22]:
query = """
SELECT course_mnemonic, COUNT(*) AS learning_outcome_count
FROM learning_outcomes
GROUP BY course_mnemonic;
"""
print(pd.read_sql(query, conn))


  course_mnemonic  learning_outcome_count
0          DS5001                       1
1          DS5100                       1
2          DS6001                       1


In [25]:
# Export courses table
pd.read_sql("SELECT * FROM courses;", conn).to_csv('courses.csv', index=False)

# Export learning outcomes table
pd.read_sql("SELECT * FROM learning_outcomes;", conn).to_csv('learning_outcomes.csv', index=False)

# Export instructors table
pd.read_sql("SELECT * FROM instructors;", conn).to_csv('instructors.csv', index=False)

# Export assignments table
pd.read_sql("SELECT * FROM assignments;", conn).to_csv('assignments.csv', index=False)
